In [15]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [16]:
def pretty_print_docs(docs):
    for i, d in enumerate(docs):
        print(f"문서 {i+1}:\n\n" + d.page_content)
        print(f"{'-' * 100}\n")

In [25]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_teddynote.document_compressors import LLMChainFilter
from langchain.retrievers import ContextualCompressionRetriever

In [18]:
from langchain_openai import ChatOpenAI

### LLM 을 활용한 문서 필터링

`LLMChainFilter`는 문서 필터링 기능을 합니다. 질문과 관련성이 있는 높은 문서들만 필터링하고 관련성이 없는 문서는 drop. <br>
이 필터는 문서 내용을 변경(압축)하지 않고 문서를 선택적으로 반환 합니다.

In [19]:
loader = TextLoader("./data/appendix-keywords.txt", encoding='utf-8')

In [20]:
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
texts = loader.load_and_split(text_splitter)

In [21]:
embeddings = OpenAIEmbeddings(api_key=key)

In [22]:
# 기본 retriever 생성
retriever = FAISS.from_documents(texts, embedding=embeddings).as_retriever()

In [23]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1
)

In [24]:
# LLM을 사용하여 LLMChainFilter 객체를 생성
_filter = LLMChainFilter.from_llm(llm)

In [26]:
# ContextualCompressionRetriever 객체를 생성
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter,        # 위에서 만든 LLMChainFilter
    base_retriever=retriever,       # 위에서 만든 retriever    
)

In [27]:
compressed_docs = compression_retriever.invoke(
    "Semantic Search 에 대해서 알려줘."
)

In [28]:
pretty_print_docs(compressed_docs)

문서 1:

Semantic Search

정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.
예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.
연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝

Embedding
----------------------------------------------------------------------------------------------------

